# 📤 FASE 5: Exportar Modelos y Datos (Community Edition)

**Objetivo:** Preparar modelos y datos para consumo via API REST

**⚠️ Community Edition:** No permite DBFS público, usamos Delta Tables

**Exporta:**
1. Modelos ya están en Delta Table `football_models` ✅
2. Datos de equipos como tabla Delta
3. Metadata para API REST

---

## 1. Verificar Modelos Existentes

In [0]:
from pyspark.sql import functions as F
import pickle
import json
import pandas as pd

# Verificar que los modelos ya están guardados
df_models = spark.table("football_models")

print("=" * 60)
print("📦 MODELOS EN DELTA TABLE")
print("=" * 60)
print(f"Total modelos: {df_models.count()}")

models_info = df_models.select("model_name", "model_type", "accuracy").toPandas()
for idx, row in models_info.iterrows():
    print(f"  ✅ {row['model_name']}")
    print(f"     Tipo: {row['model_type']}")
    print(f"     Métrica: {row['accuracy']:.3f}")

print("=" * 60)

📦 MODELOS EN DELTA TABLE
Total modelos: 3
  ✅ match_result_classifier
     Tipo: Gradient Boosting
     Métrica: 0.434
  ✅ goals_predictor
     Tipo: Gradient Boosting Regressor
     Métrica: 1.396
  ✅ goals_predictor_scaler
     Tipo: StandardScaler
     Métrica: 0.000


## 2. Crear Tabla Consolidada para API

In [0]:
# Consolidar todos los datos que necesita la API en una sola tabla
api_data = spark.table("football_team_stats") \
    .join(
        spark.table("football_team_names"),
        F.col("football_team_stats.team") == F.col("football_team_names.team_id")
    ) \
    .join(spark.table("football_team_venue_stats"), "team") \
    .select(
        F.col("football_team_stats.team").alias("team_id"),
        "team_name",
        # Stats generales
        "avg_goals_scored",
        "avg_goals_conceded",
        "win_rate",
        "avg_possession",
        "avg_shots",
        # Stats de local
        "home_avg_goals_scored",
        "home_win_rate",
        "home_advantage_goals",
        "home_advantage_points",
        # Stats de visitante
        "away_avg_goals_scored",
        "away_win_rate"
    )

# Guardar en Delta
api_data.write.format("delta").mode("overwrite").saveAsTable("football_api_data")

print("✅ Tabla API creada: football_api_data")
print(f"   Equipos: {api_data.count()}")
display(api_data.limit(5))

# Cambio principal: El join con football_team_names ahora usa:
# .join(
#     spark.table("football_team_names"),
#     F.col("football_team_stats.team") == F.col("football_team_names.team_id")
# )

✅ Tabla API creada: football_api_data
   Equipos: 25


team_id,team_name,avg_goals_scored,avg_goals_conceded,win_rate,avg_possession,avg_shots,home_avg_goals_scored,home_win_rate,home_advantage_goals,home_advantage_points,away_avg_goals_scored,away_win_rate
8,Brighton,1.8157894736842106,1.2982456140350878,0.5087719298245614,51.14473684210526,12.736842105263158,1.9298245614035088,0.6140350877192983,0.22807017543859653,0.4736842105263157,1.7017543859649122,0.40350877192982454
11,Fulham,1.1491228070175439,1.412280701754386,0.2982456140350877,45.84035087719298,10.412280701754385,1.1929824561403508,0.3508771929824561,0.08771929824561386,0.4035087719298245,1.105263157894737,0.24561403508771928
20,Southampton,1.105263157894737,1.8245614035087718,0.23684210526315788,48.11315789473684,11.605263157894736,1.2280701754385965,0.2807017543859649,0.2456140350877194,0.3157894736842106,0.9824561403508771,0.19298245614035087
2,Arsenal,1.7894736842105263,1.1403508771929824,0.5789473684210527,55.46578947368421,14.368421052631579,1.9649122807017543,0.6140350877192983,0.3508771929824561,0.2456140350877194,1.6140350877192982,0.543859649122807
12,Crystal Palace,1.5087719298245614,1.0175438596491229,0.4473684210526316,60.895614035087725,14.333333333333334,1.543859649122807,0.42105263157894735,0.07017543859649122,-0.01754385964912286,1.4736842105263157,0.47368421052631576


## 3. Crear Tabla de Feature Names

In [0]:
# Extraer feature names del modelo
model_row = df_models.filter(F.col("model_name") == "match_result_classifier").first()
feature_names = pickle.loads(model_row['feature_names'])

# Convertir a DataFrame
feature_names_df = spark.createDataFrame(
    [(i, name) for i, name in enumerate(feature_names)],
    ["feature_id", "feature_name"]
)

feature_names_df.write.format("delta").mode("overwrite").saveAsTable("football_feature_names")

print("✅ Feature names guardados en Delta Table")
print(f"   Total features: {len(feature_names)}")
display(feature_names_df)

✅ Feature names guardados en Delta Table
   Total features: 20


feature_id,feature_name
0,home_avg_goals_scored
1,home_avg_goals_conceded
2,home_win_rate
3,home_avg_possession
4,home_avg_shots
5,home_venue_goals_scored
6,home_venue_win_rate
7,home_advantage_goals
8,home_advantage_points
9,away_avg_goals_scored


## 4. Crear Tabla de Metadata

In [0]:
# Metadata del proyecto para la API
metadata = spark.createDataFrame([
    ("project_name", "AI Football Analyst"),
    ("version", "1.0"),
    ("total_teams", str(api_data.count())),
    ("total_matches", str(spark.table("football_matches_clean").count())),
    ("features_count", str(len(feature_names))),
    ("classifier_accuracy", str(models_info[models_info['model_name'] == 'match_result_classifier']['accuracy'].values[0])),
    ("regressor_mae", str(models_info[models_info['model_name'] == 'goals_predictor']['accuracy'].values[0]))
], ["key", "value"])

metadata.write.format("delta").mode("overwrite").saveAsTable("football_api_metadata")

print("✅ Metadata guardada")
display(metadata)

✅ Metadata guardada


key,value
project_name,AI Football Analyst
version,1.0
total_teams,25
total_matches,1140
features_count,20
classifier_accuracy,0.4342105263157895
regressor_mae,1.3963311637520919


## 5. Instrucciones para Cloud Run API

In [0]:
print("=" * 80)
print("📋 INSTRUCCIONES PARA CLOUD RUN API")
print("=" * 80)
print("""
La API de Cloud Run deberá:

1. Conectarse a Databricks usando SQL Warehouse (si disponible)
   O usar Databricks REST API para ejecutar queries

2. Queries necesarias:

   a) Obtener lista de equipos:
      SELECT team_id, team_name FROM football_api_data

   b) Obtener stats de un equipo:
      SELECT * FROM football_api_data WHERE team_name = '{team}'

   c) Obtener modelos:
      SELECT model_pickle FROM football_models WHERE model_name = '{name}'

   d) Obtener feature names:
      SELECT feature_name FROM football_feature_names ORDER BY feature_id

3. Alternativa (Databricks Jobs API):
   - Crear notebook que ejecuta predicción
   - Cloud Run llama al notebook via Jobs API
   - Notebook retorna resultado

4. Alternativa SIMPLE (Recomendada para Community Edition):
   - Exportar datos manualmente una vez como JSON
   - Cloud Run carga JSON en memoria (caché)
   - No necesita conexión constante a Databricks
""")
print("=" * 80)

📋 INSTRUCCIONES PARA CLOUD RUN API

La API de Cloud Run deberá:

1. Conectarse a Databricks usando SQL Warehouse (si disponible)
   O usar Databricks REST API para ejecutar queries

2. Queries necesarias:

   a) Obtener lista de equipos:
      SELECT team_id, team_name FROM football_api_data

   b) Obtener stats de un equipo:
      SELECT * FROM football_api_data WHERE team_name = '{team}'

   c) Obtener modelos:
      SELECT model_pickle FROM football_models WHERE model_name = '{name}'

   d) Obtener feature names:
      SELECT feature_name FROM football_feature_names ORDER BY feature_id

3. Alternativa (Databricks Jobs API):
   - Crear notebook que ejecuta predicción
   - Cloud Run llama al notebook via Jobs API
   - Notebook retorna resultado

4. Alternativa SIMPLE (Recomendada para Community Edition):
   - Exportar datos manualmente una vez como JSON
   - Cloud Run carga JSON en memoria (caché)
   - No necesita conexión constante a Databricks



## 6. Exportar Datos como JSON (Para descarga manual)

In [0]:
# Exportar datos a Pandas para guardar como JSON localmente
api_data_pandas = api_data.toPandas()
feature_names_pandas = feature_names_df.toPandas()
metadata_pandas = metadata.toPandas()

# Convertir a JSON (mostrar para copiar)
api_data_json = api_data_pandas.to_json(orient='records', indent=2)
feature_names_json = feature_names_pandas['feature_name'].to_json(orient='values', indent=2)
metadata_json = metadata_pandas.set_index('key')['value'].to_json(indent=2)

print("=" * 80)
print("📄 JSON GENERADOS (Copiar estos datos)")
print("=" * 80)

print("\n1️⃣ team_stats.json (primeros 3 equipos):")
print(api_data_pandas.head(3).to_json(orient='records', indent=2))

print(f"\n2️⃣ feature_names.json:")
print(feature_names_json)

print(f"\n3️⃣ metadata.json:")
print(metadata_json)

print("\n💡 Para obtener el JSON completo:")
print("   - Ejecuta: api_data_pandas.to_json(orient='records', indent=2)")
print("   - Copia el output completo")
print("   - Guarda como team_stats.json en tu proyecto Flask")

📄 JSON GENERADOS (Copiar estos datos)

1️⃣ team_stats.json (primeros 3 equipos):
[
  {
    "team_id":8,
    "team_name":"Brighton",
    "avg_goals_scored":1.8157894737,
    "avg_goals_conceded":1.298245614,
    "win_rate":0.5087719298,
    "avg_possession":51.1447368421,
    "avg_shots":12.7368421053,
    "home_avg_goals_scored":1.9298245614,
    "home_win_rate":0.6140350877,
    "home_advantage_goals":0.2280701754,
    "home_advantage_points":0.4736842105,
    "away_avg_goals_scored":1.701754386,
    "away_win_rate":0.4035087719
  },
  {
    "team_id":11,
    "team_name":"Fulham",
    "avg_goals_scored":1.149122807,
    "avg_goals_conceded":1.4122807018,
    "win_rate":0.298245614,
    "avg_possession":45.8403508772,
    "avg_shots":10.4122807018,
    "home_avg_goals_scored":1.1929824561,
    "home_win_rate":0.350877193,
    "home_advantage_goals":0.0877192982,
    "home_advantage_points":0.4035087719,
    "away_avg_goals_scored":1.1052631579,
    "away_win_rate":0.2456140351
  },
  {

## 7. Exportar Modelos como Base64 (Para Cloud Run)

In [0]:
import base64

# Función para convertir modelo pickle a base64 (transportable)
def model_to_base64(model_name):
    model_row = df_models.filter(F.col("model_name") == model_name).first()
    model_bytes = model_row['model_pickle']
    return base64.b64encode(model_bytes).decode('utf-8')

# Exportar cada modelo
print("=" * 80)
print("🔐 MODELOS EN BASE64 (Para variables de entorno)")
print("=" * 80)
print("\n⚠️ NOTA: Estos strings son muy largos, mejor descargar como pickle")
print("\nAlternativa recomendada:")
print("1. Usar queries SQL desde Cloud Run")
print("2. O exportar pickles a un bucket GCS")
print("3. O incluir en el repo (si son < 100MB)")
print("=" * 80)

🔐 MODELOS EN BASE64 (Para variables de entorno)

⚠️ NOTA: Estos strings son muy largos, mejor descargar como pickle

Alternativa recomendada:
1. Usar queries SQL desde Cloud Run
2. O exportar pickles a un bucket GCS
3. O incluir en el repo (si son < 100MB)


## 8. Resumen de Tablas Delta para API

In [0]:
print("=" * 80)
print("✅ TABLAS DELTA LISTAS PARA API")
print("=" * 80)

tables_info = [
    ("football_models", "3 modelos ML serializados"),
    ("football_api_data", "25 equipos con features completos"),
    ("football_feature_names", "20 feature names ordenados"),
    ("football_api_metadata", "Metadata del proyecto"),
    ("football_predictions_sample", "50 predicciones de ejemplo")
]

for table_name, description in tables_info:
    count = spark.table(table_name).count()
    print(f"\n  ✅ {table_name}")
    print(f"     Descripción: {description}")
    print(f"     Registros: {count}")

print("\n" + "=" * 80)

✅ TABLAS DELTA LISTAS PARA API

  ✅ football_models
     Descripción: 3 modelos ML serializados
     Registros: 3

  ✅ football_api_data
     Descripción: 25 equipos con features completos
     Registros: 25

  ✅ football_feature_names
     Descripción: 20 feature names ordenados
     Registros: 20

  ✅ football_api_metadata
     Descripción: Metadata del proyecto
     Registros: 7

  ✅ football_predictions_sample
     Descripción: 50 predicciones de ejemplo
     Registros: 50



## 9. Siguiente Paso: Flask API

In [0]:
print("=" * 80)
print("🚀 PRÓXIMO PASO: FLASK API EN CLOUD RUN")
print("=" * 80)
print("""
Opciones para implementar la API:

OPCIÓN A: DATOS ESTÁTICOS (Recomendada para Community Edition)
─────────────────────────────────────────────────────────────
1. Copiar JSONs generados arriba
2. Guardar en Cloud Run como archivos
3. API carga en memoria al iniciar (rápido)
4. No requiere conexión a Databricks
5. ✅ Funciona 100% en Community Edition

OPCIÓN B: DATABRICKS REST API
──────────────────────────────
1. Cloud Run usa Databricks REST API
2. Ejecuta queries SQL contra Delta Tables
3. Requiere token de autenticación
4. ⚠️ Puede tener latencia
5. ⚠️ Community Edition tiene límites de API

OPCIÓN C: HYBRID
────────────────
1. Datos estáticos en JSON (caché)
2. Endpoint /refresh que consulta Databricks
3. Mejor de ambos mundos
4. ✅ Recomendado para producción

DECISIÓN: Usar OPCIÓN A para el hackathon
""")
print("=" * 80)

🚀 PRÓXIMO PASO: FLASK API EN CLOUD RUN

Opciones para implementar la API:

OPCIÓN A: DATOS ESTÁTICOS (Recomendada para Community Edition)
─────────────────────────────────────────────────────────────
1. Copiar JSONs generados arriba
2. Guardar en Cloud Run como archivos
3. API carga en memoria al iniciar (rápido)
4. No requiere conexión a Databricks
5. ✅ Funciona 100% en Community Edition

OPCIÓN B: DATABRICKS REST API
──────────────────────────────
1. Cloud Run usa Databricks REST API
2. Ejecuta queries SQL contra Delta Tables
3. Requiere token de autenticación
4. ⚠️ Puede tener latencia
5. ⚠️ Community Edition tiene límites de API

OPCIÓN C: HYBRID
────────────────
1. Datos estáticos en JSON (caché)
2. Endpoint /refresh que consulta Databricks
3. Mejor de ambos mundos
4. ✅ Recomendado para producción

DECISIÓN: Usar OPCIÓN A para el hackathon



## 10. Exportar Dataset Completo (Para Flask)

In [0]:
# Crear un solo registro con toda la info necesaria
# Convertir predicciones a pandas y luego a dict (convierte dates a string)
sample_predictions_df = spark.table("football_predictions_sample").limit(10).toPandas()
sample_predictions_df['match_date'] = sample_predictions_df['match_date'].astype(str)

full_export = {
    "teams": api_data_pandas.to_dict(orient='records'),
    "feature_names": feature_names,
    "metadata": metadata_pandas.set_index('key')['value'].to_dict(),
    "sample_predictions": sample_predictions_df.to_dict(orient='records')
}

full_export_json = json.dumps(full_export, indent=2)

print("=" * 80)
print("📦 EXPORT COMPLETO PARA FLASK API")
print("=" * 80)
print(f"Tamaño del JSON: {len(full_export_json) / 1024:.2f} KB")
print(f"Equipos incluidos: {len(full_export['teams'])}")
print(f"Features: {len(full_export['feature_names'])}")
print("\n💡 Este JSON contiene TODO lo necesario para la API")
print("   Guárdalo como 'football_data.json' en tu proyecto Flask")
print("=" * 80)

# Mostrar preview
print("\nPreview (primeros 500 caracteres):")
print(full_export_json[:500] + "...")

# Cambio clave:
sample_predictions_df['match_date'] = sample_predictions_df['match_date'].astype(str)

📦 EXPORT COMPLETO PARA FLASK API
Tamaño del JSON: 17.33 KB
Equipos incluidos: 25
Features: 20

💡 Este JSON contiene TODO lo necesario para la API
   Guárdalo como 'football_data.json' en tu proyecto Flask

Preview (primeros 500 caracteres):
{
  "teams": [
    {
      "team_id": 8,
      "team_name": "Brighton",
      "avg_goals_scored": 1.8157894736842106,
      "avg_goals_conceded": 1.2982456140350878,
      "win_rate": 0.5087719298245614,
      "avg_possession": 51.14473684210526,
      "avg_shots": 12.736842105263158,
      "home_avg_goals_scored": 1.9298245614035088,
      "home_win_rate": 0.6140350877192983,
      "home_advantage_goals": 0.22807017543859653,
      "home_advantage_points": 0.4736842105263157,
      "away_avg_go...


## 11. Resumen Final

In [0]:
print("=" * 80)
print("✅ FASE 5 COMPLETADA - EXPORT PARA API")
print("=" * 80)
print("\n📊 Datos preparados:")
print("   ✅ 5 Delta Tables listas")
print("   ✅ JSONs generados para export manual")
print("   ✅ Modelos disponibles en football_models")
print("\n🎯 Próximos pasos:")
print("   1. Copiar JSONs de las celdas anteriores")
print("   2. Crear Flask API en Cloud Run")
print("   3. Deploy a GCP")
print("   4. Crear frontend en GitHub Pages")
print("\n💡 Recomendación:")
print("   Usar OPCIÓN A (datos estáticos) para el hackathon")
print("   Es más simple, rápido y funciona 100% en Community Edition")
print("=" * 80)

✅ FASE 5 COMPLETADA - EXPORT PARA API

📊 Datos preparados:
   ✅ 5 Delta Tables listas
   ✅ JSONs generados para export manual
   ✅ Modelos disponibles en football_models

🎯 Próximos pasos:
   1. Copiar JSONs de las celdas anteriores
   2. Crear Flask API en Cloud Run
   3. Deploy a GCP
   4. Crear frontend en GitHub Pages

💡 Recomendación:
   Usar OPCIÓN A (datos estáticos) para el hackathon
   Es más simple, rápido y funciona 100% en Community Edition
